In [ ]:
# Notebook styling
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

# Helping session

## 1. Customizing colorbars and colormaps in `matplotlib`

##### Question:
I have a colorbar such as this:
![](../figures/help2.JPG)

How can I get a colorbar to look like this?

![](../figures/help1.JPG)

For the plot like this:
![](../figures/help3.JPG)

### 1.0 some necessary imports and fake data

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
mpl.rcParams['figure.figsize'] = (13,9)

In [ ]:
x = np.arange(0, np.pi, 0.1)
y = np.arange(0, 2*np.pi, 0.1)
xx, yy = np.meshgrid(x, y)
zz = np.clip(6*(np.cos(xx) * np.sin(yy) + np.random.rand(*xx.shape)*np.cos(yy)),0,5.5)

### 1.1 Customizing a colorbar

Let's check if the colormap is available here: http://matplotlib.org/examples/color/colormaps_reference.html

Yes, it is available under the name 'gist_stern'

In [ ]:
fig = plt.figure(figsize=(18, 3))
ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
cb1 = mpl.colorbar.ColorbarBase(ax1, orientation='horizontal', cmap='viridis')

In [ ]:
np.linspace(0, 6, 13)

In [ ]:
fig = plt.figure(figsize=(18, 3))
ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
cb1 = mpl.colorbar.ColorbarBase(ax1, cmap='gist_stern_r',
                                boundaries=np.arange(0,6.5,0.5),
                                ticks=np.arange(0,7,1),
                                format='%1.2f',
                                orientation='horizontal')
cb1.ax.tick_params(labelsize=20)

In [ ]:
fig, ax = plt.subplots()
p = ax.contourf(x, y, zz, cmap='gist_stern_r', levels=np.arange(0,6.5,0.5))
cb1 = fig.colorbar(p, ax=ax,
                      ticks=np.arange(0,7,1),
                      format='%1.2f',
                      orientation='vertical',
                      pad=0.01)
cb1.ax.tick_params(labelsize=20)

### 1.2 Creating your own colormap

There are several ways to create your own or customize an existing colormap.

### 1.2.1 NEVER USE RAINBOW

### 1.2.2 Colormap from a list of colors

In [ ]:
cmap1 = mpl.colors.ListedColormap(['r', 'b', 'c'])
cmap1.set_over('0.25')
cmap1.set_under('0.75')

In [ ]:
fig, ax = plt.subplots()
p = ax.contourf(x, y, zz, cmap=cmap1, levels=np.arange(0,6.5,0.5),extend='max')
cb1 = fig.colorbar(p, ax=ax,
                      ticks=np.arange(0,7,1),
                      format='%1.2f',
                      orientation='vertical',
                      pad=0.01)
cb1.ax.tick_params(labelsize=20)

### 1.2.3 `LinearSegmentedColormap`

In [ ]:
c_white_trans = mpl.colors.colorConverter.to_rgba('white', alpha=0.0)
cmap = mpl.colors.LinearSegmentedColormap.from_list('aaaa',
                                                    ['#000077', c_white_trans, 'r', 'c'], 64)

In [ ]:
fig, ax = plt.subplots()
_ = ax.contourf(x, y, xx[:,::-1], cmap='viridis')
p = ax.contourf(x, y, zz, cmap=cmap, levels=np.arange(0,6.5,0.5))
cb1 = fig.colorbar(p, ax=ax,
                      ticks=np.arange(0,7,1),
                      format='%1.2f',
                      orientation='vertical',
                      pad=0.01)
cb1.ax.tick_params(labelsize=20)

In [ ]:
# references